# Results
### Load packages

In [ ]:
import plotly.graph_objects as go
import lightning.pytorch as pl
import pandas as pd
import numpy as np
import torch
import os

from pytorch_forecasting import Baseline, TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data.encoders import TorchNormalizer, GroupNormalizer
from sklearn.metrics import mean_squared_error, mean_absolute_error

### Load data

In [ ]:
Directory = 'C:/.../TFT_for_Stock_Movement_Prediction/data'
Directory = 'C:/.../TFT_for_Stock_Movement_Prediction/results'

# Target
T = pd.read_csv(os.path.join(Directory, 'CCR.csv'), index_col = [0])[750:3250].reset_index(drop = True)

# Trading feature
constituents_data = pd.read_csv(os.path.join(Directory, 'constituents_data.csv'), index_col = [0], header = [0, 1])[750:3251].reset_index(drop = True)

## Model results
# Individual
RI = pd.read_csv(os.path.join(Directory2, 'Individual/Nr.1/Results_Individual.csv'), index_col = [0])
RIp90 = pd.read_csv(os.path.join(Directory2, 'Individual/Nr.1/Results_p90_Individual.csv'), index_col = [0])

RI2 = pd.read_csv(os.path.join(Directory2, 'Individual/Nr.2/Results_Individual.csv'), index_col = [0])
RI2p90 = pd.read_csv(os.path.join(Directory2, 'Individual/Nr.2/Results_p90_Individual.csv'), index_col = [0])

# Portfolio
RP = pd.read_csv(os.path.join(Directory2, 'Global/Nr.1/Results_Global.csv'), index_col = [0])
RPp90 = pd.read_csv(os.path.join(Directory2, 'Global/Nr.1/Results_p90_Global.csv'), index_col = [0])

RP2 = pd.read_csv(os.path.join(Directory2, 'Global/Nr.2/Results_Global.csv'), index_col = [0])
RP2p90 = pd.read_csv(os.path.join(Directory2, 'Global/Nr.2/Results_p90_Global.csv'), index_col = [0])

# Sector portfolio
RPS = pd.read_csv(os.path.join(Directory2, 'Global_sector/Nr.1/Results_Global_sector.csv'), index_col = [0])
RPSp90 = pd.read_csv(os.path.join(Directory2, 'Global_sector/Nr.1/Results_p90_Global_sector.csv'), index_col = [0])

RPS2 = pd.read_csv(os.path.join(Directory2, 'Global_sector/Nr.2/Results_Global_sector.csv'), index_col = [0])
RPS2p90 = pd.read_csv(os.path.join(Directory2, 'Global_sector/Nr.2/Results_p90_Global_sector.csv'), index_col = [0])

# FNN
RMLP1 = pd.read_csv(os.path.join(Directory2, 'FNN/Nr.1/Results_FNN.csv'), index_col = [0])
RMLP2 = pd.read_csv(os.path.join(Directory2, 'FNN/Nr.2/Results_FNN.csv'), index_col = [0])

# ARIMA
RA = pd.read_csv(os.path.join(Directory2, 'ARIMA/Results_ARIMA.csv'), index_col = [0])

# Baseline
B = pd.read_csv(os.path.join(Directory2, 'Baseline/Baseline.csv'), index_col = [0])

### Define variables

In [ ]:
# Model
M = RI2
Quantile = 0.5

# Define study periods length
period_b = 0, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250
period_e = 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250
test_size = 250

# Transaction costs
tc = 0.1

### Model results

In [ ]:
## Forecast evaluation
Mmae = pd.DataFrame(index = range(len(period_b)), columns = M.columns)
Mrmse = pd.DataFrame(index = range(len(period_b)), columns = M.columns)
Mq = pd.DataFrame(index = range(len(period_b)), columns = M.columns)
b = period_b[0]
e = test_size
temp = 0
for i in range(len(period_b)):
    for j in M.columns:
        for k in range(b, e):
            temp += max(Quantile * (T[j][k] - M[j][k]), (1 - Quantile) * (M[j][k] - T[j][k]))
        Mq[j][i] = temp / test_size
        temp = 0
        Mmae[j][i] = mean_absolute_error(T[j][b:e], M[j][b:e])
        Mrmse[j][i] = mean_squared_error(T[j][b:e], M[j][b:e], squared = False)
    b += test_size
    e += test_size
print(f'Q{Quantile}: {round(sum(Mq.apply(np.sum)) / len(period_b) / len(M.columns), 4)}')
print(f'MAE: {round(mean_absolute_error(T, M), 4)}')
print(f'RMSE: {round(mean_squared_error(T, M, squared = False), 4)}')

### Trading evaluation
## Daily trading
Md = pd.DataFrame(data = np.where(M, M >= 0, 1), index = M.index, columns = M.columns)#-----------# Model
#Md = pd.DataFrame((1), index = M.index, columns = M.columns)#------------------------------------# Buy and hold
#Md = pd.DataFrame(data = np.where(T, T >= 0, 1), index = M.index, columns = M.columns)#----------# Optimal
long_r = 0
short_r = 0
for r in range(len(M)):
    temp_l = sum(T.loc[r][T.columns.intersection(Md.loc[r][Md.loc[r] == 1].index)]) / len(M.columns) / len(M)
    temp_s = sum(T.loc[r][T.columns.intersection(Md.loc[r][Md.loc[r] == 0].index)]) / len(M.columns) / len(M)
    long_r += temp_l * 100
    short_r += temp_s * 100
print('-----------------------Daily trading-----------------------')
print(f'Mean daily return: {round((long_r - short_r), 4)}%')
print(f'Mean daily return with transaction costs: {round(long_r - short_r - tc, 4)}%')

## Daily change trading evaluation
# Daily
Mdr = pd.DataFrame(index = range(len(M)), columns = M.columns)
Mdt = pd.DataFrame(index = range(len(M)), columns = M.columns)
Mdrt = pd.DataFrame(index = range(len(M)), columns = M.columns)
# Study period
Mdrp = pd.DataFrame(index = range(len(period_b)), columns = M.columns)
Mdrtp = pd.DataFrame(index = range(len(period_b)), columns = M.columns)
Mt = pd.DataFrame(index = range(len(period_b)), columns = M.columns)
Long = 0
Short = 0
for g in M.columns:
    j = 1
    k = 0
    t = 0
    for h in range(len(period_b)):
        long_r = 0
        short_r = 0
        for i in range(test_size):
            if j == period_b[h] + test_size and Md[g][j - 2] == Md[g][j - 1]:
                if Md[g][k] == 1:
                    Mdr[g][j - 1] = ((constituents_data['Close'][g][j] / constituents_data['Close'][g][k]) - 1)
                    long_r += ((constituents_data['Close'][g][j] / constituents_data['Close'][g][k]) - 1)
                else:
                    Mdr[g][j - 1] = -((constituents_data['Close'][g][j] / constituents_data['Close'][g][k]) - 1)
                    short_r += -((constituents_data['Close'][g][j] / constituents_data['Close'][g][k]) - 1)
            elif j == period_b[h] + test_size and Md[g][j - 2] != Md[g][j - 1]:
                if Md[g][j - 1] == 1:
                    Mdr[g][j - 1] = ((constituents_data['Close'][g][j] / constituents_data['Close'][g][j - 1]) - 1)
                    long_r += ((constituents_data['Close'][g][j] / constituents_data['Close'][g][j - 1]) - 1)
                else:
                    Mdr[g][j - 1] = -((constituents_data['Close'][g][j] / constituents_data['Close'][g][j - 1]) - 1)
                    short_r += -((constituents_data['Close'][g][j] / constituents_data['Close'][g][j - 1]) - 1)
            elif Md[g][k] != Md[g][j]:
                if Md[g][k] == 1:
                    Mdr[g][j - 1] = ((constituents_data['Close'][g][j] / constituents_data['Close'][g][k]) - 1)
                    long_r += ((constituents_data['Close'][g][j] / constituents_data['Close'][g][k]) - 1)
                else:
                    Mdr[g][j - 1] = -((constituents_data['Close'][g][j] / constituents_data['Close'][g][k]) - 1)
                    short_r += -((constituents_data['Close'][g][j] / constituents_data['Close'][g][k]) - 1)
                j += 1
                k = j - 1
                t += 1
            else:
                j += 1
            if i == 0:
                Mdt[g][j - 2] = -tc / 2
            elif i == test_size - 1:
                Mdt[g][j - 1] = -tc / 2
            if i == 0 and pd.isna(Mdr[g][j - 2]) == False:
                Mdt[g][j - 2] += -tc
            if i != 0 and i != test_size - 1 and pd.isna(Mdr[g][j - 2]) == False:
                Mdt[g][j - 2] = -tc
        Long += long_r / test_size * 100
        Short += short_r / test_size * 100
        Mt[g][h] = t + 1
        Mdrp[g][h] = (long_r + short_r) * 100 / test_size
        Mdrtp[g][h] = ((long_r + short_r) * 100  / test_size) - (tc * (t + 1) / test_size)
        j = 1
        k = 0
        t = 0
        j += (h + 1) * test_size
        k += (h + 1) * test_size
print('-----------------------Change trading----------------------')
print(f'Mean daily long postiton return: {round(Long / len(period_b) / len(M.columns), 4)}%')
print(f'Mean daily short postiton return: {round(Short / len(period_b) / len(M.columns), 4)}%')
print(f'Mean daily return: {round((sum(Mdr.apply(np.sum)) / len(M) / len(M.columns)) * 100, 4)}%')
print(f'Mean standard deviation: {round(sum(Mdr.apply(np.std)) / len(M.columns) * 100, 4)}%')
print(f'Mean daily transaction costs: {round(sum(Mt.apply(np.sum)) / len(M) / len(M.columns) * tc, 4)}% with {sum(Mt.apply(np.sum))} trades ({round(sum(Mt.apply(np.sum)) / len(M), 4)} trades per day)')
print(f'Mean daily return with transaction costs: {round((sum(Mdr.apply(np.sum)) / len(M) / len(M.columns)) * 100 + sum(Mdt.apply(np.sum)) / len(M) / len(M.columns), 4)}%')
Mdr = Mdr.fillna(0) * 100
Mdrt = Mdr + Mdt

### Save results

In [ ]:
# Individual
mq_Individual = (Mmae.apply(np.sum, axis = 1) / len(M.columns)).values
rp_Individual = (Mdrp.apply(np.sum, axis = 1) / len(M.columns)).values
rtp_Individual = (Mdrtp.apply(np.sum, axis = 1) / len(M.columns)).values

temp = ((Mdr).apply(np.sum, axis = 1) / len(M.columns))
r_Individual = pd.DataFrame(0, index = M.index, columns = ['Profit'])
r_Individual['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    r_Individual['Profit'][i] = r_Individual['Profit'][i - 1] + temp[i]
    
temp = ((Mdrt).apply(np.sum, axis = 1) / len(M.columns))
rt_Individual = pd.DataFrame(0, index = M.index, columns = ['Profit'])
rt_Individual['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    rt_Individual['Profit'][i] = rt_Individual['Profit'][i - 1] + temp[i]

In [ ]:
# Global
mq_Portfolio = (Mmae.apply(np.sum, axis = 1) / len(M.columns)).values
rp_Portfolio = (Mdrp.apply(np.sum, axis = 1) / len(M.columns)).values
rtp_Portfolio = (Mdrtp.apply(np.sum, axis = 1) / len(M.columns)).values

temp = ((Mdr).apply(np.sum, axis = 1) / len(M.columns))
r_Portfolio = pd.DataFrame(0, index = M.index, columns = ['Profit'])
r_Portfolio['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    r_Portfolio['Profit'][i] = r_Portfolio['Profit'][i - 1] + temp[i]
    
temp = ((Mdrt).apply(np.sum, axis = 1) / len(M.columns))
rt_Portfolio = pd.DataFrame(0, index = M.index, columns = ['Profit'])
rt_Portfolio['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    rt_Portfolio['Profit'][i] = rt_Portfolio['Profit'][i - 1] + temp[i]

In [ ]:
# Global sector
mq_Portfolio2 = (Mmae.apply(np.sum, axis = 1) / len(M.columns)).values
rp_Portfolio2 = (Mdrp.apply(np.sum, axis = 1) / len(M.columns)).values
rtp_Portfolio2 = (Mdrtp.apply(np.sum, axis = 1) / len(M.columns)).values

temp = ((Mdr).apply(np.sum, axis = 1) / len(M.columns))
r_Portfolio2 = pd.DataFrame(0, index = M.index, columns = ['Profit'])
r_Portfolio2['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    r_Portfolio2['Profit'][i] = r_Portfolio2['Profit'][i - 1] + temp[i]
    
temp = ((Mdrt).apply(np.sum, axis = 1) / len(M.columns))
rt_Portfolio2 = pd.DataFrame(0, index = M.index, columns = ['Profit'])
rt_Portfolio2['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    rt_Portfolio2['Profit'][i] = rt_Portfolio2['Profit'][i - 1] + temp[i]

In [ ]:
# FNN
mq_MLP = (Mmae.apply(np.sum, axis = 1) / len(M.columns)).values
rp_MLP = (Mdrp.apply(np.sum, axis = 1) / len(M.columns)).values
rtp_MLP = (Mdrtp.apply(np.sum, axis = 1) / len(M.columns)).values

temp = ((Mdr).apply(np.sum, axis = 1) / len(M.columns))
r_MLP = pd.DataFrame(0, index = M.index, columns = ['Profit'])
r_MLP['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    r_MLP['Profit'][i] = r_MLP['Profit'][i - 1] + temp[i]
    
temp = ((Mdrt).apply(np.sum, axis = 1) / len(M.columns))
rt_MLP = pd.DataFrame(0, index = M.index, columns = ['Profit'])
rt_MLP['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    rt_MLP['Profit'][i] = rt_MLP['Profit'][i - 1] + temp[i]

In [ ]:
# ARIMA
mq_ARIMA = (Mmae.apply(np.sum, axis = 1) / len(M.columns)).values
rp_ARIMA = (Mdrp.apply(np.sum, axis = 1) / len(M.columns)).values
rtp_ARIMA = (Mdrtp.apply(np.sum, axis = 1) / len(M.columns)).values

temp = ((Mdr).apply(np.sum, axis = 1) / len(M.columns))
r_ARIMA = pd.DataFrame(0, index = M.index, columns = ['Profit'])
r_ARIMA['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    r_ARIMA['Profit'][i] = r_ARIMA['Profit'][i - 1] + temp[i]
    
temp = ((Mdrt).apply(np.sum, axis = 1) / len(M.columns))
rt_ARIMA = pd.DataFrame(0, index = M.index, columns = ['Profit'])
rt_ARIMA['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    rt_ARIMA['Profit'][i] = rt_ARIMA['Profit'][i - 1] + temp[i]

In [ ]:
# Buy and hold
rp_Bh = (Mdrp.apply(np.sum, axis = 1) / len(M.columns)).values
rtp_Bh = (Mdrtp.apply(np.sum, axis = 1) / len(M.columns)).values

temp = ((Mdr).apply(np.sum, axis = 1) / len(M.columns))
r_Bh = pd.DataFrame(0, index = M.index, columns = ['Profit'])
r_Bh['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    r_Bh['Profit'][i] = r_Bh['Profit'][i - 1] + temp[i]
    
temp = ((Mdrt).apply(np.sum, axis = 1) / len(M.columns))
rt_Bh = pd.DataFrame(0, index = M.index, columns = ['Profit'])
rt_Bh['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    rt_Bh['Profit'][i] = rt_Bh['Profit'][i - 1] + temp[i]

In [ ]:
# Optimal
rp_Opt = (Mdrp.apply(np.sum, axis = 1) / len(M.columns)).values
rtp_Opt = (Mdrtp.apply(np.sum, axis = 1) / len(M.columns)).values

temp = ((Mdr).apply(np.sum, axis = 1) / len(M.columns))
r_Opt = pd.DataFrame(0, index = M.index, columns = ['Profit'])
r_Opt['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    r_Opt['Profit'][i] = r_Opt['Profit'][i - 1] + temp[i]
    
temp = ((Mdrt).apply(np.sum, axis = 1) / len(M.columns))
rt_Opt = pd.DataFrame(0, index = M.index, columns = ['Profit'])
rt_Opt['Profit'][0] = temp[0]
for i in range(1, len(temp)):
    rt_Opt['Profit'][i] = rt_Opt['Profit'][i - 1] + temp[i]

### Plots
#### Study periods

In [ ]:
# MAE
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
fig = go.Figure([
    go.Bar(name = 'Individual', x = x, y = mq_Individual, marker_color = 'cornflowerblue'),
    go.Bar(name = 'Global', x = x, y = mq_Portfolio, marker_color = 'palevioletred'),
    go.Bar(name = 'Global sector', x = x, y = mq_Portfolio2, marker_color = 'tan'),
    go.Bar(name = 'ARIMA', x = x, y = mq_ARIMA, marker_color = 'orange'),
    go.Bar(name = 'FNN', x = x, y = mq_MLP, marker_color = 'darkgray')])
fig.update_layout(yaxis_title = 'MAE', xaxis_title = 'Study period', plot_bgcolor = 'white', bargroupgap = 0.0, legend = dict(x = 0, y = 1.025, bgcolor = 'rgba(0,0,0,0)', orientation = 'h'))
fig.update_xaxes(ticks = 'outside', showgrid = False, ticklen = 5, dtick = 'M1')
fig.update_yaxes(gridwidth = 0.5, gridcolor = 'lightgrey')
config = {'toImageButtonOptions': {'scale': 10}}
fig.show(config = config)

In [ ]:
# Return
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
fig = go.Figure([
    go.Bar(name = 'Individual', x = x, y = rp_Individual, marker_color = 'cornflowerblue'),
    go.Bar(name = 'Portfolio', x = x, y = rp_Portfolio, marker_color = 'tan'),
    go.Bar(name = 'Sector portfolio', x = x, y = rp_Portfolio2, marker_color = 'orange')])
fig.update_layout(yaxis_title = 'Return', xaxis_title = 'Study period', plot_bgcolor = 'white', bargroupgap = 0.0, legend = dict(x = -0.05, y = 0.975, bgcolor = 'rgba(0,0,0,0)'))
fig.update_xaxes(ticks = 'outside', showgrid = False, ticklen = 5, dtick = 'M1')
fig.update_yaxes(zeroline = True, zerolinewidth = 1, zerolinecolor = 'black', gridwidth = 1, gridcolor = 'lightgrey')
config = {'toImageButtonOptions': {'scale': 2}}
for i in range(len(period_b) - 1):
    fig.add_vline(x = i + 1.5 , line_width = 1, line_dash = '7', line_color = 'lightgrey')  
fig.show(config = config)

#### Daily returns

In [ ]:
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
fig = go.Figure([
    go.Scatter(name = 'Individual', x = M.index, y = rt_Individual['Profit'], marker_color = 'cornflowerblue', line_width = 10),
    go.Scatter(name = 'Global', x = M.index, y = rt_Portfolio['Profit'], marker_color = 'palevioletred', line_width = 10),
    go.Scatter(name = 'Global sector', x = M.index, y = rt_Portfolio2['Profit'], marker_color = 'tan', line_width = 10),
    #go.Scatter(name = 'ARIMA', x = M.index, y = rt_ARIMA['Profit'], marker_color = 'orange', line_width = 10),
    #go.Scatter(name = 'FNN', x = M.index, y = rt_MLP['Profit'], marker_color = 'darkgray', line_width = 10),
    go.Scatter(name = 'Buy and hold', x = M.index, y = rt_Bh['Profit'], marker_color = 'seagreen', line_width = 10)])
fig.update_layout(yaxis_title = 'Cumulative return', xaxis_title = 'Study period', plot_bgcolor = 'white', legend = dict(x = 0, y = 1.1, bgcolor = 'rgba(0,0,0,0)', orientation = 'h', itemsizing = 'constant'))
fig.update_xaxes(ticktext = x, tickvals = M.index[125::250], ticks = 'outside', showgrid = False, ticklen = 5)
fig.update_yaxes(zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'black', gridwidth = 0.5, gridcolor = 'lightgrey')
config = {'toImageButtonOptions': {'scale': 10}}
for i in range(1, len(period_b)):
    fig.add_vline(x = period_b[i] , line_width = 0.5, line_dash = '7', line_color = 'lightgrey')  
fig.show(config = config)

### Variable importance
#### Load checkpoint

In [ ]:
best_model_path='logs/Individual/Nr.2/Period_7/Stock_17/version_0/checkpoints/epoch=41-step=420.ckpt'
#best_model_path='logs/Global/Nr.2/Period_7/version_0/checkpoints/epoch=15-step=1408.ckpt'
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

#### Select stock and study period

In [ ]:
study_period = 7
stock = 17

#### Define variables

In [ ]:
Directory = 'C:/.../TFT_for_Stock_Movement_Prediction/data'

# Target and return feature
CCR = pd.read_csv(os.path.join(Directory, 'CCR.csv'), index_col = [0])

### Features
## Time features - Categorical
time_features = pd.read_csv(os.path.join(Directory, 'time_features.csv'), index_col = [0])['0'].tolist()
for i in range(len(time_features)):
    locals()[time_features[i]] = pd.read_csv(os.path.join(Directory, time_features[i] + '.csv'), index_col = [0])

## Basic historical features
bh_features = pd.read_csv(os.path.join(Directory, 'bh_features.csv'), index_col = [0])['0'].tolist()
for i in range(len(bh_features)):
    locals()[bh_features[i]] = pd.read_csv(os.path.join(Directory, bh_features[i] + '.csv'), index_col = [0])

# Categorical
bh_categorical_features = pd.read_csv(os.path.join(Directory, 'bh_categorical_features.csv'), index_col = [0])['0'].tolist()

# Continuous
bh_continuous_features = pd.read_csv(os.path.join(Directory, 'bh_continuous_features.csv'), index_col = [0])['0'].tolist()

## Technical indicators - Continuous
indicator_features = pd.read_csv(os.path.join(Directory, 'indicator_features.csv'), index_col = [0])['0'].tolist()
for i in range(len(indicator_features)):
    locals()[indicator_features[i]] = pd.read_csv(os.path.join(Directory, indicator_features[i] + '.csv'), index_col = [0])

## Industry sectors
Sector = pd.read_csv(os.path.join(Directory, 'portfolio_table.csv'), index_col = [0])

# Study periods length
period_b = 0, 250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250
period_e = 1000, 1250, 1500, 1750, 2000, 2250, 2500, 2750, 3000, 3250

# Split period into training, validation and test set
training_size = 750
test_size = 250
validation_split = 0.2
training_cutoff = int(training_size - training_size * validation_split)

# Target
Target_feature = ['CCR']

# Features
Feature_type = ['object']
time_varying_known_categoricals = time_features
time_varying_unknown_categoricals = bh_categorical_features
time_varying_unknown_reals = bh_continuous_features + indicator_features
static_categoricals = ['Sector']

## Model parameters
# Dataset
max_prediction_length = 1
max_encoder_length = 258

# Datasets for each study period and stock
def dataset(period, stock):
    data = pd.DataFrame(index = globals()[Target_feature[0]].index[period_b[period] : period_e[period]])
    
    ## Add features
    data['Time_idx'] = range(period_b[0] + 1, period_e[0] + 1)
    data['Target'] = globals()[Target_feature[0]][[globals()[Target_feature[0]].columns[stock]]][period_b[period] : period_e[period]]
    data['Stock'] = globals()[Target_feature[0]].columns[stock]
    data['Sector'] = globals()[static_categoricals[0]]['Supersector'][stock]

    # Time varying known categoricals
    for f in range(len(time_varying_known_categoricals)):
        data[time_varying_known_categoricals[f]] = globals()[time_varying_known_categoricals[f]].astype(Feature_type[0])
    
    # Time varying unknown categoricals
    for f in range(len(time_varying_unknown_categoricals)):
        data[time_varying_unknown_categoricals[f]] = globals()[time_varying_unknown_categoricals[f]][globals()[time_varying_unknown_categoricals[f]].columns[stock]].astype(Feature_type[0])
    
    # Time varying unknown reals
    for f in range(len(time_varying_unknown_reals)):
        data[time_varying_unknown_reals[f]] = globals()[time_varying_unknown_reals[f]][globals()[time_varying_unknown_reals[f]].columns[stock]]
    return data

#### Individual

In [ ]:
data = dataset(study_period - 1, stock - 1)
training = TimeSeriesDataSet(
data[lambda x: x.Time_idx <= training_cutoff],
time_idx = data.columns[0],
target = data.columns[1],
group_ids = [data.columns[2]],
max_encoder_length = max_encoder_length,
max_prediction_length = max_prediction_length,
time_varying_known_reals = [data.columns[0]],
time_varying_unknown_reals = [data.columns[1]] + time_varying_unknown_reals,
time_varying_known_categoricals = time_varying_known_categoricals,
time_varying_unknown_categoricals = time_varying_unknown_categoricals,
target_normalizer = TorchNormalizer())
test = TimeSeriesDataSet.from_dataset(training, data, min_prediction_idx = training_size + 1)
test_dataloader = test.to_dataloader(train = False, batch_size = test_size)
raw_predictions = best_tft.predict(test, mode = 'raw', return_x = True)
interpretation = best_tft.interpret_output(raw_predictions.output, reduction = 'sum')
best_tft.plot_interpretation(interpretation)

#### Global sector

In [ ]:
stock_data_list = []
for j in range(len(locals()[Target_feature[0]].columns)):
    temp = dataset(study_period - 1, j)
    stock_data_list.append(temp)
data = pd.concat(stock_data_list, axis = 0).reset_index(drop = True)

#### Create dataframe
training = TimeSeriesDataSet(
data[lambda x: x.Time_idx <= training_cutoff],
time_idx = data.columns[0],
target = data.columns[1],
group_ids = [data.columns[2]],
max_encoder_length = max_encoder_length,
max_prediction_length = max_prediction_length,
time_varying_known_reals = [data.columns[0]],
time_varying_unknown_reals = [data.columns[1]] + time_varying_unknown_reals,
time_varying_known_categoricals = time_varying_known_categoricals,
time_varying_unknown_categoricals = time_varying_unknown_categoricals,
static_categoricals = static_categoricals,
target_normalizer = GroupNormalizer(groups = [data.columns[2]]))
test = TimeSeriesDataSet.from_dataset(training, data, min_prediction_idx = 950 + 1)
test_dataloader = test.to_dataloader(train = False, batch_size = test_size)
raw_predictions = best_tft.predict(test, mode = 'raw', return_x = True)
interpretation = best_tft.interpret_output(raw_predictions.output, reduction = 'sum')
best_tft.plot_interpretation(interpretation)

### Lighning logs

In [ ]:
from lightning.pytorch.loggers import TensorBoardLogger
%load_ext tensorboard
%tensorboard --logdir='logs/Individual/Period_1'